## Q2 Problem statement - There are times when a user writes Good, Nice App or any other positive text, in the review and gives 1-star rating. Your goal is to identify the reviews where the semantics of review text does not match rating. 


### Import Relevant libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from sklearn.linear_model import LinearRegression
import streamlit as st
import pickle
from pickle import dump
from pickle import load
import textblob

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esha.kak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
st.title('Analysing User Reviews')
st.sidebar.header('Please see the Instructions')

st.sidebar.markdown("1. The column name should be **Review/Text** ")
st.sidebar.markdown("2. The column of Rating should be **Ratings**")
st.sidebar.markdown("3. Rating should be 0to5")

2022-07-05 21:17:58.122 
  command:

    streamlit run C:\Users\esha.kak\.conda\envs\py3-TF2.0\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=1, _provided_cursor=None, _parent=DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None), _block_type=None, _form_data=None)

In [3]:
ps = PorterStemmer()

In [4]:
all_stpwords = stopwords.words('english')
all_stpwords.remove('not')

In [5]:
def text_cleaned(text):
    clean = re.sub('[^a-zA-Z]', " " , text)
    clean = clean.lower()
    clean.split()
    clean = [port.stem(word) for word in clean if word not in all_stpwords] 
    clean = ' '.join(clean)
    return clean

In [6]:
st.title("Identifying the Reviews")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [7]:
uploaded_file = st.file_uploader('Select a file')

In [8]:
data = pd.read_csv('chrome_reviews.csv')
st.write(data)

In [9]:
if st.button("Click for Results") :
    data["Cleaned_data"] = data["Text"].apply(lambda x: text_cleaned(str(x)))

    sid = SentimentIntensityAnalyzer()

    data["sentiments_vader"] = data["Text_cleaned"].apply(lambda x:sid.polarity_scores(x))
    data["Vader_Compound_Score"]  = data['sentiments_vader'].apply(lambda score_dict: score_dict['compound'])
    data["Result"] = data["Vader_Compound_Score"].apply(lambda c: 
                                                        'positive' if c > 0 else ('negative' if c < 0 else 'neutral'))
    st.bar_chart(data.Result.value_counts())

    data_positive = data[(data.Result == "positive")]
    data_positive["Opinion_Positive"] = data_positive["Star"].apply(lambda star: 
                                                                    "No Attention Needed" if star >= 3 
                                                                    else "Attention Needed")
    st.bar_chart(data_positive.Opinion_Positive.value_counts())

    data = data_positive

    st.download_button(
        label="Download data as CSV",
        df=data.to_csv().encode("utf-8"),
        file_name='data.csv',
        mime='text/csv',
    )